## Thursday 2/11/2017 : 

I started by cloning/downloading the git repository of openCitations locally and start analyzing the general structure/organization of the project. My main attention today was on the ‘website/‘ directory which defines the web pages of OpenCitations. Starting from the server module ‘oc.py’, I studied and commented the code and its main parts. This analyzes was essential to better understand the server code and web.py functionalities since I have never used it before.


Once I have understood the basic/essential parts, I re-adapted the configuration file according to my local URLs. All the necessary modules to run the python server were successfully installed under the python2.7 version. The website runs correctly locally and I can use it to formulate some tests on it.

---

## Friday 3/11/2017 :

Analyzing the .html files and generally all the static modules of the website. I have created a subdirectory inside my repository [doc/](doc/) with a *oc-rep-doc.md* file: I will use this file to write down notes and a road map on approaching some problems. Since my focus right now is on the website part, I started by adding some notes along with the adjustments that we might or eventually should do. Part of these notes are based on tests I am doing with the modification of the actual code.


One of our objects is creating a query interface which will operate on our corpus. Therfore as a possible first modification to the interface I have added a search box in the index menu, and the possible input right now for experimental reasons can be any type of text. The idea is to redirect the search operation as a GET request with a query, maybe to the sparql page.

---

## Monday 6/11/2017 :

I started reading some SPARQL documentations specially regarding the query formats to adapt for our case, giving our general textual input. I have tested different SPARQL query approaches on some external datasets. I think the idea is to apply the searching input as a filter, e.g:  `FILTER regex(?title, "SomeText”)`, in case the searching input is directed to the titles.

I have created a separated session (page) on the web interface. In this case, after typing the textual content inside the searching box, the user will be redirected to the dedicated page. The idea is to visualize the eventual results retrieved in a structured form. This page will be also designed in the future for considering advanced and additional filtering/searching parameters.

---

## Tuesday 7/11/2017:

I am trying to better understand blazegraph (https://www.blazegraph.com/) especially the the bigdata full text indexing and searching features provided. Since this might be a possible approach for general textual searching. We need to take in consideration the final performances we achieve with such approach, despite for example directing the actual queries to specific fields (e.g: DOI, Author …), after applying some filters and regular expression mapping to the actual searching input. 

I have created a python module to test the SPARQLWrapper library and functionalities. A similar python script should be called at the server side in order to perform the SPARQL query on our dataset and generate the results in JSON format to the client side. 

---

## Wednesday 8/11/2017:

I need to start doing some queries locally on my machine with a OCC data dump. My aim is to retrieve one of the initial data dumps made and uploaded on figshare (https://figshare.com/), like the September 24, 2016 dump, and test some sparql queries through blazegraph on it. I am having some technical issues regard the unpacking of all the necessary files from the .dar archives. Hopefully with the help of Silvio Peroni i will fix this problem and have all the data correctly stored locally on my machine. Meanwhile I was still testing some sparql possible queries we could possibly apply. 


I continued to deepen the blazegraph documentation, and testing its interface components/features. I am using some tutorials and samples for a better comprehension.


---

## Thursday 9/11/2017:

I managed to handle the technical problems while generating all the files needed from the .dar archives, with the help of Silvio Peroni. So right now i have the *blazegraph/sparql* endpoint running on my 3000 localhost port. I can use this sparql endpoint as a target for my queries. Through python I am testing two approaches: 1) with the Pymantic library (suggested as an example from blazegraph site). 2) SparqlWrapper.

I think SparqlWrapper is a much better solution, since its also highly used in a variety of sparql applications. 
Beside testing, I am deepen my study in the OCO list of ontologies. Trying to figure out the relations I mostly need when applying the queries.

---

## Friday 10/11/2017:

I have made a python module that should handle my regular expressions calls. For instance one important rule should detect a DOI from the textual input in the search box, in this case I studied the DOI structure/format and the possible regular expression rules which can match any given DOI. I will call this module and its inner defined methods, for any other possible textual matching problem encountered, besides the searching operations.

In [5]:
#A possible regular expression form for DOI detection:  
import re
match_obj = re.match( r'10.\d{4,9}/[-._;()/:A-Z0-9]+$', "my_text", re.M|re.I)

The textual matching phase will be a pre-step before calling the sparql endpoint and applying a query to the id field. I have tested some queries regarding how to correctly getting the actual DOI values from the bibliographic resources, and comparing those with a value given. 

---

## Monday 13/11/2017:

I have been testing some of the full text search operations through the blazegraph sparql endpoint. Basically, I have formulated queries which will search for all the literals that have a value equal to the searching field. For instance to find a particular DOI:

In [ ]:
select ?subj ?p ?lit 
where {
        ?lit bds:search "10.1002/cncr.28414" .
        ?lit bds:matchAllTerms "true" .
        ?subj ?p ?lit .  
}

We have arranged a meeting with all the DASPLAB members and discussed the possible issues and future applications. I have continued the rest of the day formulating some tests on blazegraph sparql endpoint. 

---

## Tuesday 14/11/2017:

I have created new additional modules to include inside the website subdirectory:
* sparql_conf.json : this file will contain some configuration/variables needed to the sparql_endpoint when executing the queries. (ex: the prefixes). Right now it contains the OC parameters, for instance the prefixes OCO (ontologies prefixes) this parameters could be reconfigured in case we need to manage other systems and datasets.
* ~~src/sparql_endpoint.py : this module creats the sparql endpoint object initialized with the parameters of sparql_conf.json, and it will be called for the application of queries, specially when called from the searching operations. Right now the main method in it, is ‘do_fulltext_query’ which will apply a full text search and return the results in json format.~~ 
> This approach has been changed. See **Wednesday 15/11/2017**.
* html-template/search.html, src/search.js : these 2 modules are used to build the search section/page. 

---

## Wednesday 15/11/2017:

I have started developing the actual Search/ page interface and layout (working with search.html and search.js). The layout of the page is similar to the following scheme. When applying a search operation the sparql query returns results in a json format. The json data will populate a table object, which will be visualized in the searching results container. The filters on the left side will enable a further re-elaboration on the results. The header will contain operations/visualizations to apply on the results, e.g: a sort operation.

![Image](img/search-layout.png)

**A necessary re-adaption:** I must move the sparql queries elaborations from the .py module to a .js one. This operation is necessary, since we need to build a general processing approach which might be used by any other system (Besides OC). This behavior is possible with the addition of a re-configurable json file which defines all the configurations and possible queries.

---

## Thursday 16/11/2017 :

In order to let the searching procedure be re-adaptable for any system, I have built a .json file containing the parameters and configurations variables which could be defined by the user: 
* sparql_endpoint
* a list of the prefixes (to add before the actual querie)
* a list of rules: each rule is defined by three parameters: the id, the regex rule and the corresponding textual query to apply. Inside the query we insert a VAR tag which will be replaced with the searching text.

I have defined a .js method to call sparql and apply the query, along with .js method that handles the regular expressions and assign the correct query, as its defined in the rules field inside the configuration file.
I am having some issues when calling sparql through .js, since it seems i can't get .json result when calling a sparql endpoint which resides on a different domain (the server should be allowed to do that: *Access-Control-Allow-Origin*). Therefor the possible solution is to retrieve the GET result in a .jsonp format, which still is not giving me the expected results.

---

## Friday 17/11/2017:

I have managed to solve yesterday problem (the Access-Control-Allow-Origin issue) with the help of Silvio: The OC server already allows the calls on external origins, therefore the solution was to redirect all the GET calls to the server which will further apply a query to the dataset.
Now the procedure to accomplish through javascript is almost all defined. Any search call will go through this steps/methods :   
* 1\. **_get_search_conf():** to get all the search configurations, the sparql endpoind url, the prefixes and the rules (see Thursday)
* 2\. Build the query string in turtle format:
	* 2.1\. **turtle_prefixes():** builds a string combining all the prefixes of the config file, in a turtle format (*PREFIX [prefix.name] : <[prefix.url]>* ) 
	* 2.2\. **turtle_query(text):** to accomplish this operation we go through this steps:
		* 2.2.1\.  iterate the list of rules and check which regex rule matches the *text* parameter 
		* 2.2.2\.  return the corresponding string of the query in turtle format and replace the *VAR* tag inside it with *text*. (e.g: if the query looks for a DOI, the variable *text* to look for could be equal to '10.1002/cncr.28414' )    
* 3\. **do_search_query(query_str):** apply the query and get the results in json format.
* 4\. **build_tab_from_json(json_data):** builds a structured table for the results obtained from the previous step.

---

## Monday 20/11/2017 :

I have finalized the components and methods described yesterday, specially the graphic structure of the table. The current system can successfully elaborate all the searching operations in case we have a defined matching rule inside the configuration file. Here is a possible example scheme for a DOI search:

<img src="img/query-ex.png" style="height: 450px; width: 620px;"/>

We have arranged our second DASPLAB meeting and discussed interesting future works [doc/recap-DASPLAB](doc/recap-DASPLAB). I have added other rules to handle ORCID and general text (e.g: names, titles) searches. The next step is building and integrating the filtering operations into the search section.

---

## Tuesday 21/11/2017:

Today I managed to implement some filters and visualization parameters to apply on the query results:
* Maximum number of results: the total number of results retrieved from the query will not exceed this number. This value could be chosen from the user through a slider input.
* Number of results per page: the maximum number of rows we can visualize in a page. The possible values are: 5,10,20,30,40,50. The user can switch between these values from a dropdown list. In order to navigate through the pages I have added 2 buttons at the bottom of the table: next and previous.

In addition I have redesigned the visual interface and applied the new visual elements on it. Here a snapshot of the search section:

<img src="img/snap_search.png" style="height: 450px; width: 680px;"/>

**Note:** all the filters and visualization parameters applied will not involve the use of sparql queries. 

---

## Wednesday 22/11/2017:

I started by fixing some issues and uncaught problems from yesterday, along with improving the features already developed. The next step is to add new filters and parameters to let users apply them on the results:
* Results relevance: this value could be specified within the sparql query. The dynamic re-arrange of this value from the filtering section should not make a sparql query again, so a user will have the ability only to check a new value smaller than the original score.
* Fields values: we will have the ability to filter the fields (columns) of our results and view only the rows with the values we are interested in. e.g: if the results are a list of documents, we can filter and view only those of the year "2013".

I have started to develop the second point. To handle this aspect, a user must specify the category of the expected results from each different rule within the search configuration file. Along with this for each category we will list the fields that we wish to filter. e.g: the rule that matches the DOIs retrieves results in DOCUMENT category, this category will have fields like TITLE, YEAR ... etc, and we might for instance choose to apply filters only to the YEAR field.

---

## Thursday 23/11/2017:

I am proceeding in the integration of filters. For each different field (column) in the table of results I have collected the distinct values. A user can choose to filter and view only some of these values or exclude the selected values from the results. Therefor i have developed a switch button that will permit the inclusion or exclusion of the values checked from the filter list. This snap-shot shows this particular section:

<img src="img/filters.png" style="height: 300px; width: 250px;"/>

For instance in this case a user can check the box *2013*, and then decide whether to include or exclude all the table rows which have the value *2013* for the field *year*. **Note:** all the values in the filter section are being updated dynamically according to the corresponding values in the table of results.

---

## Friday 24/11/2017:

I have finalized the filtering operations, and improved the usability of the searching interface. An additional visualization tool needed is the ability to order the results obtained. Therefor another input box has been added to the interface above the results. The user will have the ability to sort the results according to the columns in the table.

All the main searching operations have been integrated and the system is almost ready to elaborate the general text queries. The next step is formulating all the correct sparql queries needed for OC case, and retrieving the wanted information.

---

## Monday 27/11/2017:

I am studying the set of sparql queries needed to correctly retrieve all the fields we want for our major two categories of results: Documents and Authors. The one I am formulating now takes any free text as input and retrieves results as Document / s.

We have arranged our third DASPLAB meeting and discussed the current system and additional interesting future works [doc/recap-DASPLAB](doc/recap-DASPLAB). This will include additional processing of the code in order to make it as adaptable as possible for other systems. This will involve more parametrization inside the .js and .css modules.

---

## Tuesday 28/11/2017:

The main focus of the day was on the search interface and layout. I fixed some issues we have discussed and pointed out in the DASPLAB meeting of yesterday. The most notable:
* Making additional components flexible from a stylistic and behavior point of view. (e.g: What colors to assign, and the order on which components should be listed)
* Adjust the table cells and their textual contents.
* Settle correctly the html components in the filter section (at the left of the table)

Next I need to implement a dropdown list for each field in the filter section. This is duo to the fact, that keeping all the checkboxes fixed under each field will reduce the readability and the all around user experience. 

---

## Wednesday 29/11/2017:

I am testing the searching interface in order to detect both the conceptual and visual problems. I managed to handle many issues and continued the adjustment of the layout components. One of the main components I have added is a navigation index for the pages and the bottom of the table, as we can see from this snapshot:

<img src="img/page-index.png" style="height: 75px; width: 420px;"/>

This index will display the current page along with the previous and the next pages. This will enable the user to go directly to the wanted page, without using the ‘next’ and ‘previous’ buttons only. 
The second important element to add is the dynamic droplist in the filter menu. I have started working on this and I will continue working on it tomorrow also.

---

## Thursday 30/11/2017:

Today I managed to implement and integrate the a dynamic droplist for each field in the filter section. This part will adapt to some additional parameters defined in the configuration file: e.g: the min/max number of elements to display once the droplist window is open. The final result is represented in this snapshot:

<img src="img/filter-field-0.png" style="display: inline-block; height: 200px; width: 170px;"/>
Once I open the droplist &#8594;
<img src="img/filter-field-1.png" style="display: inline-block; height: 380px; width: 170px;"/>

I have started refactoring the code and adjusting some minor issues/redundant code portions, my particular attention is on the .js module at the moment. I will move also to the .css modules. This part is essential in order to let future users and developers control and comprehend correctly the code.

---

## Friday 1/12/2017 :

Another useful component has been integrated to the droplist implemented yesterday: navigation arrows at the bottom of the list. This will allow the user to scroll through all the other available checkboxes entries. Other additional parameters have been integrated in the configuration file to allow this and other graphical adaptions. 

A crucial operation is to associate the table entries to the original resources (href link).To allow this, the user needs to specify in the configuration file the corresponding column which contains the URI, in case we don’t have any URI for such field, the uri parameter in the configuration file will stay blank (or remove completely the entry).
The key-parameter to add to the field entry in the configuration file is the *link*. Which will contain 3 main parameters: *field*: specifies the column that contains its uri, *suffix*: a textual content to add at the end of the uri string, *active*: a bool value to specify whether the link is active. Here we show an example for the *field= "title"* : 

In [ ]:
{"value":"title", "type": "text","filter":true, "sort":true, "link":{"field":"doc","suffix":"","active": true}}

---

## Monday 4/12/2017 :

In addition to associating the URI link for some fields in the results, we need to group by the rows. This operation is needed in order to concatenate more rows values under one table entry only. For instance, we might have several rows for the same bibliographic resource for each different author, in this case we will group by the results and concatenate the author field in one table cell. 

We can perform this operation only from the .js module, this is due to the fact that performing a group by from the sparql endpoint will discard other information associated in other columns, for example the ‘linking’ operation previously described will not be applicable anymore and we will lose the links of each different author. We can summarize the .js operations done on the results obtained from sparql as follow:

*Data_from_sparql* &#8608; **[associate links]** &#8608; **[group by]** &#8608; **[keep the columns i want]** &#8608; *Data_for_client*

We have arranged another DASPLAB ([doc/recap-DASPLAB](doc/recap-DASPLAB)) meeting where I have introduced the system status and interface. We have discussed some system arrangements and the next steps to apply. The most notable:
* Create a different set of parameters in the config file for each different field in the filter section.
* Create an entry page for the search section.
* Formulate correctly the sparql queries

I have accomplish the first two points, and I have started refactoring the code. I will also move to the third point starting from tomorrow.

---

## Tuesday 5/12/2017:

I have dedicated my today work on the code refactoring of the .js module. I have organized the .js file in a much suitable and comprehensive form, and removed many redundant and unnecessary code. Right now I have 3 different modules:
* **util:** it will contain all the utility functions which operate and do general elaborations on the data (regard the context). e.g: *group_by()*, *sort_json_by_key()* .
* **htmldom:** in this module all the functions will create and return the html doms. e.g: *pages_prev_btn()*, *nav_btns_filter()*.
* **search:** its the main original module and will contain only the model (logical behaviors) of the system. e.g: *do_sparql_query*, *select_filter_field*.

---

## Wednesday 6/12/2017:

Today I moved my attention on the .css module, and continued my code refactoring operations on it. I have established a much comprehensive structured skeleton for the .html components and their corresponding css classes and styles. The following scheme summarizes the current html skeleton:

<img src="img/skeleton-html.png" style="display: inline-block; height: 300px; width: 670px;"/>

All these components have fixed sizes and positions, and for each element there is a corresponding css class called the same way (the underscore turns into '-'. e.g: *search_header* is associated to the css class *search-header*)

Additional internal html DOMS are built dynamically from the .js elaborations. Those additional components will follow under their corresponding div and inherit the .css parameters from the preceding levels. This form is meant to simplify the customization of the interface.  

---

## Thursday 7/12/2017:

I continued editing the .css classes for the searching DOM components of the search page. I have built the main searching page that will appear when moving directly to the searching page without applying any query (through the input box on the top of the page). Once a query is lunched a loading panel will appear until the results are ready to be visualized.

Now I moved my attention to another crucial visualization: the resources browsers. The idea is to create a more comprehensive visualization which will include all the information related to the resource browsed. I started by analyzing the part of the *LinkedDataDirector .py* module that will populate the data needed and send it to the dedicated web rendering page. I am having some issues in moving the tests on a local small corpus, I will fix this point and move on editing the current .html file.

---

## Monday 11/12/2017:

I have moved my attention to the resource browser. before building the .html components. I took some time understanding the Link-Data-Director and moving the tests on my local corpus. Then I started building the general html layout and corresponding css style.

We have arranged another DASPLAB meeting and discussed my last week work and the future elaboration steps for the browsing page. The browser is conceived as an alternative view to the current traditional rdf resource browser, and the system should guarantee both visualizations.

I have created 4 different new files: browser-cont.json, browser.js, browser.html, and browser.css. I will follow the same scheme and strategies applied for the search page.

---

## Tuesday 12/12/2017:

I am developing the browser page and its corresponding modules defined yesterday. I have accomplished some crucial and basic behaviours needed, from both the logical and graphical point of view. The document visualization in the browser is almost done and the current situation is represented in this snapshot: 

<img src="img/browser-doc.png" style="display: inline-block; height: 380px; width: 850px;"/>

In this case the sparql query will be directed to a precise resource, e.g: https://w3id.org/oc/corpus/br/1. Although we can specify the fields needed as a result inside the query, some additional operations should be handled from the .js module, e.g: group by. I will move my attention also to the author browser section starting from tomorrow.   

---

## Wednesday 13/12/2017:

The browser page is much more dynamical and it’s much more customizable directly from the configuration file. The “content” field inside the configuration file will enable the user to customize and decide the elements to visualize inside the browser page for both the categories: documents (corpus/br/*), authors (corpus/ra/*). Each entry inside the *content* field will represent a table row, and the parameters to define it are: 
* The data: a specific column/s from the query results.
* Table cell type: *td* or *th*
* CSS class: to define the style of the contents 
* Title prefix: a textual prefix before the actual data value
* Show title: a true/false on whether to show the title prefix
* String concatenation style: if the column represent an array of values (as a result of a group by operation) we can choose how to concatenate the elements: on a single line, or each element on a different line.

For instance an entry example is: 

`{"fields": ["label"], "tag":"td", "class":"", "title":[""], "show_title":false}`

I have defined the sparql queries for both the documents and the authors inside the configuration file of the browser page. Therefor the browser page can currently generate the pages of both categories. To facilitate and test these operations I am using some python modules defined locally. For instance, I am using a python module to convert a query inside a text file to a format suitable to be integrated in the configuration file.

---

## Thursday 14/12/2017:

I started by fixing some issues regarding the sparql queries in the browser configuration file. I have built a link association also to the DOI and ORCID values. Currently all the essential information for both documents and authors are retrieved and correctly visualized. Although, some additional adjustments must be applied:

* Author documents: for each author we need to visualize the list of his related documents. A possible idea is to re-use the table of results used in the search section with the application of an author query.
* Apply a value mapping for some fields visualized. For instance: http://purl.org/spar/fabio/Expression -> doc.
* Make the configuration file usable also in case we want to query other type of documents or contributors.

I moved back to the *search* section to apply some modifications, mainly in the configuration file and in the sparql queries definition. I will keep testing this section for future adaptions.
I helped the DASPLAB colleagues on the annotation of some data.

---

## Friday 15/12/2017:

While testing and correcting the sparql queries in the search section I noticed the fact that applying a search with any textual input is not retrieving all the results wanted: this is due to the fact that this type of query takes in consideration only the reference entry for the matching operations, therefor this will exclude all the other resources (/br/) that are not cited and have no ref-entry. 
I have been trying to figure out a solution via sparql, this one seems to retrieve the correct results:

In [ ]:
SELECT DISTINCT ?doc ?doi ?title ?year ?author ?author_iri (COUNT(distinct ?cited) AS ?num_cited)
WHERE  { 

Full text search :

In [ ]:
#.
    ?lit bds:search 'THE QUERY' . 
    ?lit bds:matchAllTerms 'true' . 
    ?lit bds:relevance ?score .

Check the type of the [`?lit`] var: title, subtitle or familyName. The union betwwen the results obtained will give us all the possible results.

In [ ]:
#.
    {?iri dcterms:title  ?lit } UNION {?iri fabio:hasSubtitle ?lit} UNION {?iri foaf:familyName ?lit} .

Get all the fields needed from a bib resource (all as `OPTIONAL` values):

In [ ]:
#.    
    OPTIONAL {
        ?iri rdf:type ?type .
        BIND(?iri as ?doc) .
        OPTIONAL {?doc dcterms:title  ?title .}
        OPTIONAL {?doc fabio:hasSubtitle  ?subtitle .}
        OPTIONAL {?doc fabio:hasPublicationYear ?year .}
        OPTIONAL {?doc cito:cites ?cited .}
        OPTIONAL {
            ?iri datacite:hasIdentifier [
                datacite:usesIdentifierScheme datacite:doi ;
                literal:hasLiteralValue ?doi 
            ]
        }
  
        OPTIONAL {
             ?iri pro:isDocumentContextFor [
                 pro:withRole pro:author ;
                 pro:isHeldBy ?author_iri 
             ].
             ?author_iri foaf:familyName ?fname .
             ?author_iri foaf:givenName ?name .
             BIND(CONCAT(STR(?name),' ', STR(?fname)) as ?author) .
        }
    

All the previous values will be retrieved if the document iri analyzed is a br/ type.

In [ ]:
#.        
        FILTER regex(str(?iri), '^https://w3id.org/oc/corpus/br/')
    }

The final 2 blocks will be applied also for the /ra/ case (lit is a familyName), so in this case the filter will look like:

In [ ]:
FILTER regex(str(?iri), '^https://w3id.org/oc/corpus/ra/')

And the entry to the main OPTIONAL will try to retrieve the br/ iri giving the agent family name, so in this case we will write:

In [ ]:
OPTIONAL {
    ?iri rdf:type ?type .
    OPTIONAL{
        ?role pro:isHeldBy ?iri .
        ?doc pro:isDocumentContextFor ?role . 
        #... GET ALL THE DOCUMENT FIELDS: title, subtitle, year ... etc  

---

## Monday 18/12/2017:

The previous sparql query defined for any textual input search is inefficient, this is mostly caused by the FILTER command and a redundancy in some part of the query (applying same queries for the '/br/' resources although they only defer in the type of iri), therefore I have defined another query. 
The initial ?iri variable will take any possible value with the application of a tables Union:

In [ ]:
  {
    ?entry c4o:hasContent ?lit . 
    ?entry biro:references ?iri .
  }
  UNION
  {?iri dcterms:title  ?lit } 
  UNION 
  {?iri fabio:hasSubtitle ?lit} 
  UNION 
  {
    ?myra foaf:familyName ?lit .
	?role pro:isHeldBy ?myra .
	?iri pro:isDocumentContextFor ?role .
  } 
  .

The second part will extract all the values from the bib resource iri, this part is  same as in the previous query. 
I started working also on other general aspects of the website. An important issue is related to the fact that we need to make the site responsive and adaptable also for different screen resolutions. The navigator menu element need to be adapted for this case. I have started and almost done developing a solution to it: the full navigator menu list will disappear, and a small menu dropdown will appear instead:

<img src="img/navmenu.png" style="display: inline-block; height: 380px; width: 520px;"/>

---

## Tuesday 19/12/2017:

Instead of developing the responsive navigator menu through javascript I will use some bootstrap features. This is basically done by specifying a *”data-toggle”* and a *data-target* parameter inside the button that will contain the collapsed data (*data-target*). In order to do this I needed to readapt the html objects correctly and redefine the .css classes inside the “cover.css” file. Currently the page will readapt according to the resolution width as is shown in this screen shot:

<img src="img/responsive0.png" style="display: inline-block; height: 200px; width: 350px;"/>
<img src="img/responsive1.png" style="display: inline-block; height: 380px; width: 200px;"/>
<img src="img/responsive2.png" style="display: inline-block; height: 380px; width: 200px;"/>

There is still some small stylistic issues that need to be fixed.

---

## Wednesday 20/12/2017:

I have fixed other minor issues in the responsive navigator and in the .css file, later I started testing the website, specially the search section. I found some little problems in the search.js which I have successfully fixed. The main adjustment was applied on the filtering panel, this panel need to be updated according to the search results and the context. The iri link inside the table has been changed to a lighter version, since the initial part of it “https://w3id.org/oc/corpus” is the same for any entry of the table.

---

## Thursday 21/12/2017:

I continued testing the search.js and refining its stylistic aspect along with its behavior. With the help of Silvio we have uploaded this version to the opencitations server. After fixing and adapting the new configurations the site was set up and ready to run. Although we noticed some issues that need to be done:
* Graphical adaptions in .css in case the website is browsed through mozilla. (colors and sort buttons)
* The last sort option chosen need to remain active even when we filter the page or make any other change in the view.

I have been able to fix these two points successfully, I will do some additional attempts and send the new files modified tomorrow.

---

## Friday 22/12/2017:

Computing additional tests and interactions with the interface revealed new issues that need to be fixed. After obtaining some results from the application of the field values filter (checkboxes), these results didn’t behave correctly when choosing a different number of results from the range slider, this also lead to a wrong sort of the data in the table, so its turned out to be a major trigger for other problems. This issue and related unexpected results were also acknowledged to me by Silvio. 

Basically this problem happened due to a bad separation between the data viewed and the data filtered. I have been able to fix this along with all its related sub-issues. I am planning to do other verification tests before uploading these new modifications.  

I will create a separated git repository for the search interface (with all its related files .css/.js/.html), this will make it easier the acknowledgment of future issues/comments.

---

## Wedensday 27/12/2017:

I noticed a new issue in the search module when searching for a ORCID, since the system did not return any result. To fix the problem i needed to add some additional checks in case some field where undefined in the configuration file. 

On the browser interface I have been able to successfully integrate the search table as a part of it, in order to show the list of documents that a given author has worked on. The browser needs to call the search and apply a new query that I have defined inside the search configuration file: the query expects the author uri and retrieves all the related documents. To allow the queries defined in the search configuration accept also url variables (to put inside the query), we need to write <URL-VAR> (instead of <VAR> which expects a string). 
I continued applying other modifications on the .css .js modules of the browser.

**Note** : to apply the modifications on the server, we need to copy the *search.css, search.js, search-conf.js* from the repository. (https://github.com/ivanhb/search.git)

---
## Thursday 28/12/2017:

I am working on the browser interface and modifying/defining new features inside: browser.css, browser.js and browser-cont.js. 
The creation of the general interface and data to visualize is much more intuitive and simple and it could all be done from the configuration file.
For each different category (document or author) the user can decide which contents to visualize inside each of the different 3 sections: header, details, and metrics. Each different entry in the configuration file represent a row inside the interface, here we have an entry example:

In [ ]:
{"fields": ["FREE-TEXT","num_docs","FREE-TEXT"], "values": ["Author of ",""," documents"], "classes": ["metric-entry","imp-value"],  "tag":"td"}

* **fields :** is an array of the elements to add, in case we want to add a textual content not related to the query, we use the key *FREE-TEXT*
* **values :** the order of the values must follow the fields order, the elements inside the added row will be visualized following this order. In case the corresponding index in *fields* is a *FREE-TEXT* key, we can write our desired label, for keys extracted from the query we just leave an empty value.
* **classes :** we can define different .css classes for each field.

An important new feature included its the *textual-mapping*. Inside the configuration file we can define a mapping for some special textual occurrences. For example the document type value need to be replaced with a more comprehensive format, to allow this I have defined this entry inside the configuration file: 

In [ ]:
"text_mapping": {
            "short_type":[
                {"regex": /Expression/g, "value":"Document"},
                {"regex": /([a-z])([A-Z])/g, "value":"$1 $2"}
            ]
        }

The above *regex* rules will be applied for the *short_type* field retrieved from the query. In this case the first rule will replace all the “Expression” occurrences with “Document”. The second rule will create a space between two different words by detecting the capital letter, e.g: JournalArticle -> Journal Article. 

In some cases we would like to retrieve only the last/first element of an array of elements previously built using group-by. For example the document type will give us both “Expression” and “JournalArticle”, but the value we are interested in is just the last one. To enable this we need to assign “last” in the “concat_style” field in the configuration file. 

In [ ]:
"concat_style":{"short_type": "last"} 

---
## Friday 29/12/2017:

I have created a browser repository which can work separately from the openCitations project (https://github.com/ivanhb/browser.git). I will follow the same strategy adopted for the search repository case, therefor to apply the modifications on the server, we need to copy the browser.css, browser.js, and browser-conf.js from the repository, we also need to be careful and modify the “sparql_endpoint” inside the browser-conf file (same as the search module case).

Other graphical adaptions have been made and right now the interface looks like the following 2 snapshots (for /br and /ra):

<img src="img/browser-br.png" style="display: inline-block; height: 300px; width: 620px;"/>

<img src="img/browser-ra.png" style="display: inline-block; height: 300px; width: 620px;"/>